In [3]:
# from utils import *
import random
import Graph as G
import numpy as np


# Get vertices from DB and insert them into the graph
def Initialize_Random_Graph(DBPath='dataset/DB1K.csv',R=5):
    DBGraph=G.Graph()
    with open(DBPath, 'r') as f:
        DB = f.readlines()
        DB = [x.strip() for x in DB]
        for i,x in enumerate(DB):
            x = x.split(',')
            x= np.array(x,dtype=float)
            DBGraph.add_vertex(G.Vertex(int(x[0]),x[1:]))
            # Graph.Insert(x[0], x[1], x[2])
        # Choose R random neighbors for each vertex
        size= len(DBGraph.verticies)
        if(size==0 or size==1):
            return
        for vertex in DBGraph:
            for i in range(R):
                neighbor= DBGraph.get_vertex(int(random.random()*size))
                while(neighbor==vertex):
                    neighbor= DBGraph.get_vertex(int(random.random()*size))
                DBGraph.add_edge((vertex.key,vertex.value),(neighbor.key,neighbor.value))
    return DBGraph


#gets euclidean distance between 2 vectors
def get_distance(v1,v2):
    return  np.linalg.norm(v1-v2)


#gets medoid of a graph
def get_medoid(DBGraph):
    min_distance=10000000000000000000
    medoid=None
    for vertex in DBGraph:
        current_total_distance=0
        for vertex2 in DBGraph:
            if(vertex==vertex2):
                continue
            dist=get_distance(vertex.value,vertex2.value)
            current_total_distance+=dist
        
        if(current_total_distance<min_distance):
            min_distance=current_total_distance
            medoid=vertex
    return medoid,min_distance



#gets arg min distance from any vertex in Anyset to Query and min dist
def get_min_dist (Anyset,Query):
    min_dist=10000000000000000000
    min_vertex=None
    for vertex in Anyset:
        # print("vertex",vertex)
        # print("Query",Query[:3])
        dist=get_distance(vertex.value,Query)
        if(dist<min_dist):
            min_dist=dist
            min_vertex=vertex
    return min_vertex,min_dist


#initially, start is the medoid
# s is a vertex, Query is a vector
# k is a number, L is a number
def Greedy_Search(start,Query,k,L):
    search_List={start}
    Visited=set()
    possible_frontier=search_List #.difference(Visited)
    while possible_frontier != set():
        # print("QUERY",Query)
        # print("possible_frontier",possible_frontier)
        p_star,_= get_min_dist(possible_frontier,Query)
        #
        # print('pstar',p_star)
        # if p_star==None:
        #     # break
        #     print('frontier: ')
        #     for v in possible_frontier:
        #         print(v)
        #     print(possible_frontier==set())
        search_List=search_List.union(p_star.neighbors)
        Visited.add(p_star)
        if(len(search_List)>L):
            #update search list to retain closes L points to x_q
            search_ListL_L=list(search_List)
            search_ListL_L.sort(key=lambda x: get_distance(x.value,Query))
            # only maintain L closest points
            search_ListL_L=search_ListL_L[:L]
            search_List=set(search_ListL_L)

        possible_frontier=search_List.difference(Visited)
        
    search_ListL_L=list(search_List)
    search_ListL_L.sort(key=lambda x: get_distance(x.value,Query))
    # only maintain k closest points
    search_ListL_L=search_ListL_L[:k]
    search_List=set(search_ListL_L)
    return search_List,Visited

# Robust pruning 
def Robust_Prune(point,candidate_set,alpha,R):
    # print(candidate_set)
    candidate_set=candidate_set.union(point.neighbors)
    candidate_set.difference({point}) # changed
    point.neighbors=set()
    while candidate_set!=set():
        # print("candidate_set", candidate_set)
        # print("point.value", point.value)

        p_star,min_dist= get_min_dist(candidate_set,point.value)
        point.neighbors.add(p_star)
        if(len(point.neighbors)==R):
            break
        DummySet=candidate_set.copy()
        for candidatePoint in candidate_set:
            if(alpha * get_distance(p_star.value,candidatePoint.value)<=get_distance(candidatePoint.value,point.value)):
                DummySet.remove(candidatePoint)
        candidate_set=DummySet


def Build_Index(datasetPath, R, L):

    # R = min(R, len(dataset))

    index=Initialize_Random_Graph(datasetPath,R)
    medoid,min_distance=get_medoid(index)

    # CHECK
    randIndex = list(index.get_vertices())
    random.shuffle(randIndex)

    # random permutation + sequential graph update
    for n in randIndex:
        node = index.get_vertex(n)
        # print(n)

        (_, V) = Greedy_Search(medoid, node.value, k = 1, L = L)
        Robust_Prune(node, V, alpha= 1, R= R)
        neighbors = node.get_neighbors()
        
        for inb in neighbors:
            # nbr = index[inb]
            # nbr = index.get_vertex(inb)
            
            # CHECK : The backward edge is always added
            inb.add_neighbor(node)
            if len(inb.get_neighbors()) > R:
                # print("inb.get_neighbors()", inb.get_neighbors())
                Robust_Prune(inb, inb.get_neighbors(), alpha= 1, R= R)
                # nbr[1].add(n)

In [4]:
R=5
k=10
L=10

path='dataset/DB1K.csv'

# DBGraph=Initialize_Random_Graph(path,R)
# # print(DBGraph)
# medoid,min_distance=get_medoid(DBGraph)

Build_Index(path, R, L)

#how to choose query? how to choose k,L?

self.neighbors set()
neighbor 538 neighbors: [] 
 values: [0.69308615 0.07689482 0.44399798]
self.neighbors {<Graph.Vertex object at 0x00000271A3B63BD0>}
neighbor 276 neighbors: [] 
 values: [0.1644997  0.83918524 0.42087716]
self.neighbors {<Graph.Vertex object at 0x00000271A3B4BE90>, <Graph.Vertex object at 0x00000271A3B63BD0>}
neighbor 156 neighbors: [] 
 values: [0.31981981 0.82950699 0.31456161]
self.neighbors {<Graph.Vertex object at 0x00000271A3B4BE90>, <Graph.Vertex object at 0x00000271A3B63BD0>, <Graph.Vertex object at 0x00000271A3B556D0>}
neighbor 269 neighbors: [] 
 values: [0.59503728 0.96570158 0.05964965]
self.neighbors {<Graph.Vertex object at 0x00000271A3B4BE90>, <Graph.Vertex object at 0x00000271A3B63BD0>, <Graph.Vertex object at 0x000002718C7D5190>, <Graph.Vertex object at 0x00000271A3B556D0>}
neighbor 365 neighbors: [] 
 values: [0.09034663 0.41006023 0.30511469]
self.neighbors set()
neighbor 439 neighbors: [] 
 values: [0.15522265 0.10596198 0.7612626 ]
self.neighbo

In [46]:
alpha=2
rng = np.random.default_rng(50)
print('dasatince')
for p in medoid.neighbors:
    print(get_distance(p.value,medoid.value))
Query=rng.random((1, 70), dtype=np.float32)
# def Greedy_Search(start,Query,k,L):
SL,Vis=Greedy_Search(medoid,Query,k,L)
print(medoid)
Robust_Prune(medoid,Vis,alpha,R)
print(medoid)
for p in medoid.neighbors:
    print(get_distance(p.value,medoid.value))

dasatince
3.4806914079997227
3.0229934261642657
3.4194758402928827
3.0647342939684052
3.20948603492922
QUERY [[0.7813332  0.78742266 0.7145687  0.8336693  0.8237809  0.54790443
  0.24254471 0.97344905 0.8289076  0.23683351 0.27866644 0.6469233
  0.67119896 0.06502604 0.9596958  0.5555776  0.01293612 0.40235746
  0.6543596  0.26596576 0.6883337  0.9866823  0.6262581  0.39731765
  0.18344492 0.32845616 0.6601709  0.12515694 0.99986154 0.11145777
  0.29785854 0.98457336 0.55596834 0.5789395  0.45138943 0.8920926
  0.53687006 0.01508015 0.78798324 0.59969866 0.1192258  0.4202239
  0.54847825 0.37373084 0.9542997  0.70650905 0.27028292 0.06140804
  0.542406   0.23580968 0.91909045 0.24576241 0.6833456  0.44861054
  0.22162598 0.3316726  0.462358   0.7717461  0.15772003 0.47309273
  0.58490956 0.7804164  0.4965189  0.12486297 0.46206492 0.57882965
  0.24902534 0.70675725 0.6981578  0.6835448 ]]
possible_frontier {<Graph.Vertex object at 0x000002961B46A010>}
QUERY [[0.7813332  0.78742266 0.71